In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'entropy', 'max_depth': 7}
----------
[[ 1297  3655]
 [    6 13405]]
----------
              precision    recall  f1-score   support

           0       1.00      0.26      0.41      4952
           1       0.79      1.00      0.88     13411

    accuracy                           0.80     18363
   macro avg       0.89      0.63      0.65     18363
weighted avg       0.84      0.80      0.75     18363

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.12815726, 0.14959931, 0.18201375, 0.20345652, 0.18001842,
        0.22340167, 0.22888744, 0.26878142, 0.26279795, 0.26628745,
        0.29371405, 0.31216466, 0.32413244, 0.34358025, 0.39793444,
        0.13663316, 0.19049001, 0.19398069, 0.23038316, 0.2717725 ,
        0.28274298, 0.33560228, 0.3510617 , 0.35006356, 0.40092707,
        0.40790796, 0.45727634, 0.4812125 , 0.61635184, 0.65674508]),
 'std_fit_time': array([6.48248196e-03, 1.49610043e-02, 1.04715824e-02, 2.09437609e-02,
        4.99725342e-04, 3.57627869e-07, 1.44621134e-02, 3.49068642e-03,
        1.24675035e-02, 3.57627869e-07, 8.47697258e-03, 2.19413042e-02,
        6.98208809e-03, 3.24139595e-02, 3.98945808e-02, 7.15255737e-07,
        1.99437141e-03, 1.49583817e-03, 1.19678974e-02, 1.49500370e-03,
        1.04718208e-02, 2.04455853e-02, 2.19408274e-02, 9.97066498e-04,
        1.89495087e-02, 1.19683743e-02, 2.24397182e-02, 3.49056721e-03,
        1.79514885e-02, 4.48739529e-03]),
 'mean_scor